In [1]:
import numpy as np
import matplotlib.pyplot as plt
import PIL
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import random
from torch.autograd import Variable  # 导入torch中Variable模块

In [2]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

True


In [3]:
#HParams超参数类
class HParams():
    def __init__(self):
        self.data_location = './cat.npz'
        self.enc_hidden_size = 256#encoder
        self.dec_hidden_size = 512#decoder
        
        self.Nz = 128#latent vector size
        self.input_size = 5#输入数据的特征维数[sqe_length,batch_size,input_size],h0,c0        
        self.M = 20#高斯混合数
        self.dropout = 0.1
        self.batch_size = 128
        self.eta_min = 0.01#最小学习率
        self.R = 0.99995 # KL annealing decay rate per minibatch
        self.KL_min = 0.2# Level of KL loss at which to stop optimizing for KL
        self.wKL = 1# KL weight of loss equation. Recommend 0.5 or 1.0.
        #0.5
        self.lr = 0.001
        self.lr_decay = 0.9999# Learning rate decay per minibatch.
        self.min_lr = 0.00001
        
        self.grad_clip = 1.0 # Gradient clipping. Recommend leaving at 1.0.
        self.temperature = 0.4
        self.max_seq_length = 200
hp = HParams()

## load dataset

In [4]:
def get_max_len(strokes):
    """Return the maximum length of an array of strokes."""
    max_len = 0
    max_len = 0
    for stroke in strokes:
        ml = len(stroke)
        if ml > max_len:
            max_len = ml
    return max_len

In [5]:
def calculate_normalizing_scale_factor(strokes):
    """Calculate the normalizing factor explained in appendix of sketch-rnn."""
    data = []
    for i in range(len(strokes)):
        for j in range(len(strokes[i])):
            data.append(strokes[i][j, 0])
            data.append(strokes[i][j, 1])
    data = np.array(data)
    return np.std(data)

In [6]:
#预处理：确定范围，以最大seq长度筛选，除以scale factor
def preprocess(strokes):
    """Remove entries from strokes having > max_seq_length points."""
    data = []
    scale_factor = calculate_normalizing_scale_factor(strokes)
    for seq in strokes:
        if seq.shape[0] <= hp.max_seq_length :
            seq = np.minimum(seq, 1000)#limit=1000
            seq = np.maximum(seq, -1000)
            seq = np.array(seq, dtype=np.float32)
            seq[:,0:2]/=scale_factor
            data.append(seq)
    return data

In [7]:
def normalize(strokes):
    """Normalize entire dataset (delta_x, delta_y) by the scaling factor."""
    data = []
    scale_factor = calculate_normalizing_scale_factor(strokes)
    for seq in strokes:
        seq[:, 0:2] /= scale_factor
        data.append(seq)
    return data

In [8]:
dataset = np.load(hp.data_location, encoding='latin1',allow_pickle=True)
data = dataset['train']
data = preprocess(data)
Nmax = get_max_len(data)
Nmax

129

In [9]:
#generate a batch:
#随机抽取data的index，组成batchsize个为一组的batch，并由 x y 0/1转化为x y
#0 0 1
#返回batch和每个batch的点位数量序列
#batch_size为100，在dim=1堆叠, size(200,100,5)
def make_batch(batch_size):
    batch_idx = np.random.choice(len(data),batch_size)
    batch_sequences = [data[idx] for idx in batch_idx]#batch序列 大小为batch_size
    strokes = []
    lengths = []
    indice = 0
    for seq in batch_sequences:
        len_seq = len(seq[:,0])
        new_seq = np.zeros((Nmax,5))#返回batch和每个batch的点位数量序列.zeros((Nmax,5))
        new_seq[:len_seq,:2] = seq[:,:2]
        new_seq[:len_seq-1,2] = 1-seq[:-1,2]#0->1 0 0 1->0 1 0
        new_seq[:len_seq,3] = seq[:,2]
        new_seq[(len_seq-1):,4] = 1
        new_seq[len_seq-1,2:4] = 0
        lengths.append(len(seq[:,0]))#总长度
        strokes.append(new_seq)#笔画数据集
        indice += 1
    #转化为tensor
    batch = Variable(torch.from_numpy(np.stack(strokes,1)).cuda().float())
    return batch, lengths


In [10]:
################################ adaptive lr
def lr_decay(optimizer):
    """Decay learning rate by a factor of lr_decay"""
    for param_group in optimizer.param_groups:
        if param_group['lr']>hp.min_lr:
            param_group['lr'] *= hp.lr_decay
    return optimizer


## MODEL

In [11]:
class sketchRNN_encoder(nn.Module):
    def __init__(self):
        super(sketchRNN_encoder,self).__init__()
        self.lstm = nn.LSTM(5,hp.enc_hidden_size,bidirectional=True)
        self.dropout = nn.Dropout(hp.dropout)
        #使用全连接层从最后输出h获取向量μ和σ，大小为Nz
        self.mu_function = nn.Linear(2*hp.enc_hidden_size,hp.Nz)
        self.sigma_function= nn.Linear(2*hp.enc_hidden_size,hp.Nz)
        
        self.train()
    def forward(self,inputs,batch_size):
        #零初始化
        h_0 = torch.zeros(2,batch_size,hp.enc_hidden_size).cuda()#bi-LSTM隐含状态
        cell_0 = torch.zeros(2,batch_size,hp.enc_hidden_size).cuda()#bi-SLTM单元状态
        h_cell_0 = (h_0,cell_0)
        _,(h,cell) = self.lstm(inputs.float(),h_cell_0)
        h_forward,h_backward = torch.split(self.dropout(h),1,0)#h_forward,h_backward[2,batchsize,hiddensize]->h[batchsize,2*hidden_size]
        h_final = torch.cat([h_forward.squeeze(0),h_backward.squeeze(0)],1)
        
        #计算μ和σ
        mu = self.mu_function(h_final)
        sigma_hat = self.sigma_function(h_final)
        sigma = torch.exp(sigma_hat/2.)
        #计算z
        z_size = mu.size()
        N = torch.normal(torch.zeros(z_size),torch.ones(z_size)).cuda()#N~N(0,1)
        z = mu + sigma*N#z=μ+σN
        
        return z,mu,sigma_hat

In [12]:
class sketchRNN_decoder(nn.Module):
    def __init__(self):
        super(sketchRNN_decoder, self).__init__()
        self.lstm = nn.LSTM(hp.Nz+5, hp.dec_hidden_size)
        self.dropout = nn.Dropout(hp.dropout)
        # 从z初始化h&cell
        self.h_cell_function = nn.Linear(hp.Nz, 2*hp.dec_hidden_size)
        #初始化隐藏状态yi,yi=Wy hi + by,R^(6M+3)
        self.yi_params_function = nn.Linear(hp.dec_hidden_size,6*hp.M+3)
    def forward(self,inputs,z,h_cell_0=None):
        
        if h_cell_0 is None:
            h_0,cell_0 = torch.split(torch.tanh(self.h_cell_function(z)),hp.dec_hidden_size,1)
            h_cell_0 = (h_0.unsqueeze(0).contiguous(), cell_0.unsqueeze(0).contiguous())
        outputs,(h,cell) = self.lstm(inputs, h_cell_0)
        
        #判断是train还是eval
        #训练时使用最后一层输出特征outputs，验证使用最后一层隐藏层h
        if self.training:
            y = self.yi_params_function(outputs.view(-1,hp.dec_hidden_size))
        else:
            #eval()
            y = self.yi_params_function(self.dropout(h).view(-1,hp.dec_hidden_size))
        
        params = torch.split(y,6,1)
        params_mixture = torch.stack(params[:-1]) # trajectory
        params_pen = params[-1]
        # 获取各结果
        pi,mu_x,mu_y,sigma_x,sigma_y,rho_xy = torch.split(params_mixture,1,2)
        # preprocess params::
        if self.training:
            len_out = Nmax+1
        else:
            len_out = 1
        pi = F.softmax(pi.transpose(0,1),dim=1).view(len_out,-1,hp.M)#[len_out,？，M]
        mu_x = mu_x.transpose(0,1).squeeze().contiguous().view(len_out,-1,hp.M)
        mu_y = mu_y.transpose(0,1).squeeze().contiguous().view(len_out,-1,hp.M)
        sigma_x = torch.exp(sigma_x.transpose(0,1).squeeze()).view(len_out,-1,hp.M)
        sigma_y = torch.exp(sigma_y.transpose(0,1).squeeze()).view(len_out,-1,hp.M)
        rho_xy = torch.tanh(rho_xy.transpose(0,1).squeeze()).view(len_out,-1,hp.M)
        q = F.softmax(params_pen,dim=1).view(len_out,-1,3)#k={1,2,3}
        return pi,mu_x,mu_y,sigma_x,sigma_y,rho_xy,q,h,cell
    

In [13]:
class sketchRNN_model():
    def __init__(self):
        self.encoder = sketchRNN_encoder().cuda()
        self.decoder = sketchRNN_decoder().cuda()
        
        self.encoder_optimizer = optim.Adam(self.encoder.parameters(),hp.lr)
        self.decoder_optimizer = optim.Adam(self.decoder.parameters(),hp.lr)
        self.eta_step = hp.eta_min#最小学习率
        
    def make_target(self,batch,lengths):
        #初始化Si为【0,0,0,0,1】，增加一个维度
        eos = torch.stack([torch.Tensor([0,0,0,0,1])]*batch.size()[1]).cuda().unsqueeze(0)
        #eos（1,100,5）
        batch = torch.cat([batch,eos],0)
        #batch (201,100,5)
        mask = torch.zeros(Nmax+1, batch.size()[1])
        #mask（201，100）
        
        for indice,length in enumerate(lengths):
            #点位数序列
            #解决RNN中输入有多种长度的问题，
            #将所有有数值的地方填1，RNN跳过所有mask为0的输入
            mask[:length,indice] = 1
        mask = mask.cuda()
        
        dx = torch.stack([batch.data[:,:,0]]*hp.M,2)
        dy = torch.stack([batch.data[:,:,1]]*hp.M,2)
        p1 = batch.data[:,:,2]
        p2 = batch.data[:,:,3]
        p3 = batch.data[:,:,4]
        p = torch.stack([p1,p2,p3],2)
        return mask,dx,dy,p
    
    def train(self, epoch):
        self.encoder.train()
        self.decoder.train()
        batch, lengths = make_batch(hp.batch_size)
        ##返回batch和每个batch的点位数量序列
        #(200,100,5)
        
        # encode
        z, self.mu, self.sigma = self.encoder(batch, hp.batch_size)
        
        # create start of sequence（sos）:拓展一维(1,100,5)
        sos = torch.stack([torch.Tensor([0,0,1,0,0])]*hp.batch_size).cuda().unsqueeze(0)
        
        # had sos at the begining of the batch:
        #按行拼接(201,100,5)
        batch_init = torch.cat([sos, batch],0)
        # expend z to be ready to concatenate with inputs:
        z_stack = torch.stack([z]*(Nmax+1))#(201,[z])
        # inputs is concatenation of z and batch_inputs
        #加在dim=2
        inputs = torch.cat([batch_init, z_stack],2)
        ##########################################
        
        # decode:
        self.pi, self.mu_x, self.mu_y, self.sigma_x, self.sigma_y, \
            self.rho_xy, self.q, _, _ = self.decoder(inputs, z)
        # prepare targets:
        
        mask,dx,dy,p = self.make_target(batch, lengths)
        
        # prepare optimizers:
        self.encoder_optimizer.zero_grad()
        self.decoder_optimizer.zero_grad()
        
        # update eta for LKL:
        self.eta_step = 1-(1-hp.eta_min)*hp.R
        # compute losses:
        LKL = self.kullback_leibler_loss()
        LR = self.reconstruction_loss(mask,dx,dy,p,epoch)
        loss = LR + LKL
        
        # gradient step
        loss.backward()
        
        # gradient cliping
        nn.utils.clip_grad_norm_(self.encoder.parameters(), hp.grad_clip)
        nn.utils.clip_grad_norm_(self.decoder.parameters(), hp.grad_clip)
        
        # optim step
        self.encoder_optimizer.step()
        self.decoder_optimizer.step()
        # some print and save:
        if epoch%1==0:
            print('epoch',epoch,'loss',format(loss.item(),'.4f'),'LR',format(LR.item(),'.4f'),'LKL',format(LKL.item(),'.4f'))
            self.encoder_optimizer = lr_decay(self.encoder_optimizer)
            self.decoder_optimizer = lr_decay(self.decoder_optimizer)
        if epoch%100==0:
            #self.save(epoch)
            self.conditional_generation(epoch)

    def bivariate_normal_pdf(self, dx, dy):
        #求出双变量归一化pdf
        #tf_2d_normal
        z_x = ((dx-self.mu_x)/self.sigma_x)**2
        z_y = ((dy-self.mu_y)/self.sigma_y)**2
        z_xy = (dx-self.mu_x)*(dy-self.mu_y)/(self.sigma_x*self.sigma_y)
        z = z_x + z_y -2*self.rho_xy*z_xy
        exp = torch.exp(-z/(2*(1-self.rho_xy**2)))
        norm = 2*np.pi*self.sigma_x*self.sigma_y*torch.sqrt(1-self.rho_xy**2)
        return exp/norm

    def reconstruction_loss(self, mask, dx, dy, p, epoch):
        pdf = self.bivariate_normal_pdf(dx, dy)
        #equation9 epsilon=1e5
#         result1 = torch.sum(self.pi*pdf,2)
#         result1 = torch.mul(self.pi,pdf)
#         result1 = torch.sum(result1,1,keepdim=True)
# #         result2 = torch.sum(result)
#         result1 = -torch.log(1e-6+result1)
#         mask = torch.reshape(mask,[-1,1])
#         LS = torch.mul(mask,result1)/float(Nmax*hp.batch_size)
#         LS = torch.sum(mask*result1)/float(Nmax*hp.batch_size)
#         LS = -torch.sum(mask*torch.log(1e-5+torch.mul(self.pi * pdf))/float(Nmax*hp.batch_size)
        LS = -torch.sum(mask*torch.log(1e-6+torch.sum(self.pi * pdf, 2)))\
            /float(Nmax*hp.batch_size)
        #sum(self.pi * pdf, 2)：对应dim=2维度相加，相当于每行跨列相加求和
        LP = -torch.sum(p*torch.log(self.q))/float(Nmax*hp.batch_size)
        LR = LS+LP
        LR = torch.mean(LR)
#         print("LS:",LS," LP:",LP)
        return LR

    def kullback_leibler_loss(self):
        LKL = -0.5*torch.sum(1+self.sigma-self.mu**2-torch.exp(self.sigma))\
            /float(hp.Nz*hp.batch_size)
        KL_min = Variable(torch.Tensor([hp.KL_min]).cuda()).detach()
        return hp.wKL*self.eta_step * torch.max(LKL,KL_min)
    #self.kl_cost = tf.maximum(self.kl_cost, self.hps.kl_tolerance)

    def save(self, epoch):
        sel = np.random.rand()
        torch.save(self.encoder.state_dict(), \
            'encoderRNN_sel_%3f_epoch_%d.pth' % (sel,epoch))
        torch.save(self.decoder.state_dict(), \
            'decoderRNN_sel_%3f_epoch_%d.pth' % (sel,epoch))

    def load(self, encoder_name, decoder_name):
        saved_encoder = torch.load(encoder_name)
        saved_decoder = torch.load(decoder_name)
        self.encoder.load_state_dict(saved_encoder)
        self.decoder.load_state_dict(saved_decoder)

    def conditional_generation(self, epoch):
        batch,lengths = make_batch(1)
        # batchsize = 1
        # should remove dropouts:
        self.encoder.train(False)
        self.decoder.train(False)
        # encode:
        z, _, _ = self.encoder(batch, 1)
        # batchsize = 1

        sos = Variable(torch.Tensor([0,0,1,0,0]).view(1,1,-1).cuda())
        s = sos
        seq_x = []
        seq_y = []
        seq_z = []
        hidden_cell = None
        for i in range(Nmax):
            input = torch.cat([s,z.unsqueeze(0)],2)
            # decode:
            self.pi, self.mu_x, self.mu_y, self.sigma_x, self.sigma_y, \
                self.rho_xy, self.q, hidden, cell = self.decoder(input, z, hidden_cell)
            hidden_cell = (hidden, cell)
            # sample from parameters:
            s, dx, dy, pen_down, eos = self.sample_next_state()
            #------
            seq_x.append(dx)
            seq_y.append(dy)
            seq_z.append(pen_down)
            if eos:
                print(i)
                break
        # visualize result:
        x_sample = np.cumsum(seq_x, 0)
        y_sample = np.cumsum(seq_y, 0)
        z_sample = np.array(seq_z)
        sequence = np.stack([x_sample,y_sample,z_sample]).T
        make_image(sequence, epoch)

    def sample_next_state(self):

        def adjust_temp(pi_pdf):
            pi_pdf = np.log(pi_pdf)/hp.temperature
            pi_pdf -= pi_pdf.max()
            pi_pdf = np.exp(pi_pdf)
            pi_pdf /= pi_pdf.sum()
            return pi_pdf

        # get mixture indice:
        pi = self.pi.data[0,0,:].cpu().numpy()
        pi = adjust_temp(pi)
        pi_idx = np.random.choice(hp.M, p=pi)
        # get pen state:
        q = self.q.data[0,0,:].cpu().numpy()
        q = adjust_temp(q)
        q_idx = np.random.choice(3, p=q)
        # get mixture params:
        mu_x = self.mu_x.data[0,0,pi_idx]
        mu_y = self.mu_y.data[0,0,pi_idx]
        sigma_x = self.sigma_x.data[0,0,pi_idx]
        sigma_y = self.sigma_y.data[0,0,pi_idx]
        rho_xy = self.rho_xy.data[0,0,pi_idx]
        x,y = sample_bivariate_normal(mu_x,mu_y,sigma_x,sigma_y,rho_xy,greedy=False)
        next_state = torch.zeros(5)
        next_state[0] = x
        next_state[1] = y
        next_state[q_idx+2] = 1
        if use_cuda:
            return Variable(next_state.cuda()).view(1,1,-1),x,y,q_idx==1,q_idx==2
        else:
            return Variable(next_state).view(1,1,-1),x,y,q_idx==1,q_idx==2

In [14]:
def sample_bivariate_normal(mu_x,mu_y,sigma_x,sigma_y,rho_xy, greedy=False):
    # inputs must be floats
    if greedy:
      return mu_x,mu_y
    mean = [mu_x, mu_y]
    sigma_x *= np.sqrt(hp.temperature)
    sigma_y *= np.sqrt(hp.temperature)
    cov = [[sigma_x * sigma_x, rho_xy * sigma_x * sigma_y],\
        [rho_xy * sigma_x * sigma_y, sigma_y * sigma_y]]
    x = np.random.multivariate_normal(mean, cov, 1)
    return x[0][0], x[0][1]

def make_image(sequence, epoch, name='_output_'):
    """plot drawing with separated strokes"""
    strokes = np.split(sequence, np.where(sequence[:,2]>0)[0]+1)
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    for s in strokes:
        plt.plot(s[:,0],-s[:,1])
    canvas = plt.get_current_fig_manager().canvas
    canvas.draw()
    pil_image = PIL.Image.frombytes('RGB', canvas.get_width_height(),
                 canvas.tostring_rgb())
    name = str(epoch)+name+'.jpg'
    pil_image.save(name,"JPEG")
    plt.close("all")




In [15]:
if __name__=="__main__":
    model = sketchRNN_model()
    for epoch in range(50001):
        model.train(epoch)

    '''
    model.load('encoder.pth','decoder.pth')
    model.conditional_generation(0)
    #'''


epoch 0 loss 2.5350 LR 2.5329 LKL 0.0020
4
epoch 1 loss 2.4254 LR 2.4234 LKL 0.0020
epoch 2 loss 2.3512 LR 2.3492 LKL 0.0020
epoch 3 loss 2.3851 LR 2.3830 LKL 0.0020
epoch 4 loss 2.3245 LR 2.3224 LKL 0.0020
epoch 5 loss 2.1063 LR 2.1043 LKL 0.0020
epoch 6 loss 2.0701 LR 2.0681 LKL 0.0020
epoch 7 loss 2.1095 LR 2.1075 LKL 0.0020
epoch 8 loss 2.2341 LR 2.2321 LKL 0.0020
epoch 9 loss 2.0296 LR 2.0276 LKL 0.0020
epoch 10 loss 1.8773 LR 1.8753 LKL 0.0020
epoch 11 loss 1.8531 LR 1.8511 LKL 0.0020
epoch 12 loss 1.8231 LR 1.8211 LKL 0.0020
epoch 13 loss 1.8801 LR 1.8781 LKL 0.0020
epoch 14 loss 1.6428 LR 1.6408 LKL 0.0020
epoch 15 loss 1.6390 LR 1.6370 LKL 0.0020
epoch 16 loss 1.7140 LR 1.7120 LKL 0.0020
epoch 17 loss 1.5738 LR 1.5718 LKL 0.0020
epoch 18 loss 1.6121 LR 1.6101 LKL 0.0020
epoch 19 loss 1.5828 LR 1.5808 LKL 0.0020
epoch 20 loss 1.6875 LR 1.6855 LKL 0.0020
epoch 21 loss 1.5661 LR 1.5641 LKL 0.0020
epoch 22 loss 1.5928 LR 1.5908 LKL 0.0020
epoch 23 loss 1.5965 LR 1.5944 LKL 0.0020


epoch 197 loss 0.6995 LR 0.6894 LKL 0.0101
epoch 198 loss 0.7312 LR 0.7212 LKL 0.0101
epoch 199 loss 0.7322 LR 0.7220 LKL 0.0102
epoch 200 loss 0.6975 LR 0.6871 LKL 0.0104
epoch 201 loss 0.7011 LR 0.6908 LKL 0.0103
epoch 202 loss 0.6780 LR 0.6678 LKL 0.0102
epoch 203 loss 0.7687 LR 0.7583 LKL 0.0104
epoch 204 loss 0.7157 LR 0.7050 LKL 0.0107
epoch 205 loss 0.7329 LR 0.7219 LKL 0.0110
epoch 206 loss 0.7601 LR 0.7492 LKL 0.0109
epoch 207 loss 0.6550 LR 0.6442 LKL 0.0107
epoch 208 loss 0.6800 LR 0.6692 LKL 0.0108
epoch 209 loss 0.6970 LR 0.6864 LKL 0.0106
epoch 210 loss 0.6837 LR 0.6734 LKL 0.0103
epoch 211 loss 0.6609 LR 0.6509 LKL 0.0100
epoch 212 loss 0.7082 LR 0.6980 LKL 0.0102
epoch 213 loss 0.6683 LR 0.6583 LKL 0.0101
epoch 214 loss 0.6803 LR 0.6701 LKL 0.0103
epoch 215 loss 0.6934 LR 0.6831 LKL 0.0102
epoch 216 loss 0.7266 LR 0.7164 LKL 0.0101
epoch 217 loss 0.6410 LR 0.6309 LKL 0.0100
epoch 218 loss 0.7145 LR 0.7045 LKL 0.0100
epoch 219 loss 0.6821 LR 0.6721 LKL 0.0100
epoch 220 l

epoch 389 loss 0.5513 LR 0.5378 LKL 0.0135
epoch 390 loss 0.5236 LR 0.5100 LKL 0.0136
epoch 391 loss 0.5478 LR 0.5343 LKL 0.0135
epoch 392 loss 0.4817 LR 0.4682 LKL 0.0135
epoch 393 loss 0.5117 LR 0.4980 LKL 0.0137
epoch 394 loss 0.5672 LR 0.5535 LKL 0.0138
epoch 395 loss 0.4484 LR 0.4343 LKL 0.0141
epoch 396 loss 0.5261 LR 0.5121 LKL 0.0139
epoch 397 loss 0.5014 LR 0.4872 LKL 0.0142
epoch 398 loss 0.5175 LR 0.5035 LKL 0.0140
epoch 399 loss 0.4978 LR 0.4838 LKL 0.0139
epoch 400 loss 0.5149 LR 0.5009 LKL 0.0141
epoch 401 loss 0.5804 LR 0.5662 LKL 0.0142
epoch 402 loss 0.5095 LR 0.4952 LKL 0.0143
epoch 403 loss 0.4982 LR 0.4838 LKL 0.0144
epoch 404 loss 0.5386 LR 0.5245 LKL 0.0141
epoch 405 loss 0.4861 LR 0.4721 LKL 0.0140
epoch 406 loss 0.5091 LR 0.4950 LKL 0.0141
epoch 407 loss 0.4999 LR 0.4857 LKL 0.0142
epoch 408 loss 0.5280 LR 0.5139 LKL 0.0142
epoch 409 loss 0.5748 LR 0.5605 LKL 0.0144
epoch 410 loss 0.5212 LR 0.5070 LKL 0.0142
epoch 411 loss 0.4850 LR 0.4707 LKL 0.0143
epoch 412 l

epoch 583 loss 0.4286 LR 0.4125 LKL 0.0162
epoch 584 loss 0.4411 LR 0.4247 LKL 0.0164
epoch 585 loss 0.3906 LR 0.3745 LKL 0.0161
epoch 586 loss 0.4174 LR 0.4013 LKL 0.0161
epoch 587 loss 0.4089 LR 0.3930 LKL 0.0160
epoch 588 loss 0.4163 LR 0.4003 LKL 0.0160
epoch 589 loss 0.4446 LR 0.4288 LKL 0.0158
epoch 590 loss 0.3950 LR 0.3794 LKL 0.0156
epoch 591 loss 0.4741 LR 0.4584 LKL 0.0156
epoch 592 loss 0.4697 LR 0.4542 LKL 0.0155
epoch 593 loss 0.3848 LR 0.3692 LKL 0.0156
epoch 594 loss 0.4462 LR 0.4308 LKL 0.0154
epoch 595 loss 0.4237 LR 0.4081 LKL 0.0157
epoch 596 loss 0.3787 LR 0.3630 LKL 0.0157
epoch 597 loss 0.4456 LR 0.4298 LKL 0.0158
epoch 598 loss 0.4173 LR 0.4014 LKL 0.0159
epoch 599 loss 0.4453 LR 0.4292 LKL 0.0161
epoch 600 loss 0.4486 LR 0.4323 LKL 0.0164
epoch 601 loss 0.4141 LR 0.3976 LKL 0.0165
epoch 602 loss 0.4260 LR 0.4098 LKL 0.0162
epoch 603 loss 0.4842 LR 0.4679 LKL 0.0163
epoch 604 loss 0.4277 LR 0.4113 LKL 0.0164
epoch 605 loss 0.4320 LR 0.4155 LKL 0.0164
epoch 606 l

epoch 775 loss 0.3385 LR 0.3211 LKL 0.0174
epoch 776 loss 0.3929 LR 0.3753 LKL 0.0175
epoch 777 loss 0.3728 LR 0.3552 LKL 0.0176
epoch 778 loss 0.3457 LR 0.3285 LKL 0.0172
epoch 779 loss 0.3880 LR 0.3706 LKL 0.0174
epoch 780 loss 0.3533 LR 0.3361 LKL 0.0173
epoch 781 loss 0.3247 LR 0.3073 LKL 0.0174
epoch 782 loss 0.3794 LR 0.3620 LKL 0.0174
epoch 783 loss 0.4013 LR 0.3840 LKL 0.0173
epoch 784 loss 0.3534 LR 0.3360 LKL 0.0174
epoch 785 loss 0.3876 LR 0.3700 LKL 0.0176
epoch 786 loss 0.3344 LR 0.3169 LKL 0.0175
epoch 787 loss 0.3156 LR 0.2981 LKL 0.0175
epoch 788 loss 0.3290 LR 0.3114 LKL 0.0175
epoch 789 loss 0.3445 LR 0.3272 LKL 0.0173
epoch 790 loss 0.3626 LR 0.3453 LKL 0.0173
epoch 791 loss 0.3358 LR 0.3185 LKL 0.0174
epoch 792 loss 0.4098 LR 0.3924 LKL 0.0174
epoch 793 loss 0.3832 LR 0.3656 LKL 0.0176
epoch 794 loss 0.3670 LR 0.3496 LKL 0.0173
epoch 795 loss 0.3558 LR 0.3382 LKL 0.0175
epoch 796 loss 0.3691 LR 0.3515 LKL 0.0176
epoch 797 loss 0.3833 LR 0.3656 LKL 0.0178
epoch 798 l

...

epoch 7667 loss -0.0567 LR -0.0815 LKL 0.0248
epoch 7668 loss -0.0577 LR -0.0827 LKL 0.0250
epoch 7669 loss -0.0397 LR -0.0646 LKL 0.0248
epoch 7670 loss -0.0154 LR -0.0403 LKL 0.0249
epoch 7671 loss -0.0442 LR -0.0689 LKL 0.0248
epoch 7672 loss -0.0107 LR -0.0355 LKL 0.0249
epoch 7673 loss -0.0362 LR -0.0612 LKL 0.0250
epoch 7674 loss -0.0569 LR -0.0821 LKL 0.0251
epoch 7675 loss -0.0902 LR -0.1156 LKL 0.0254
epoch 7676 loss -0.0944 LR -0.1192 LKL 0.0249
epoch 7677 loss 0.0098 LR -0.0151 LKL 0.0249
epoch 7678 loss 0.0329 LR 0.0078 LKL 0.0251
epoch 7679 loss -0.0390 LR -0.0640 LKL 0.0250
epoch 7680 loss -0.0485 LR -0.0736 LKL 0.0250
epoch 7681 loss -0.0929 LR -0.1182 LKL 0.0253
epoch 7682 loss -0.0443 LR -0.0692 LKL 0.0249
epoch 7683 loss -0.0752 LR -0.1002 LKL 0.0250
epoch 7684 loss -0.1310 LR -0.1562 LKL 0.0252
epoch 7685 loss -0.0611 LR -0.0867 LKL 0.0255
epoch 7686 loss -0.0676 LR -0.0927 LKL 0.0251
epoch 7687 loss -0.0373 LR -0.0624 LKL 0.0251
epoch 7688 loss -0.0210 LR -0.0461 LK

epoch 7847 loss -0.0744 LR -0.0995 LKL 0.0251
epoch 7848 loss -0.0526 LR -0.0775 LKL 0.0249
epoch 7849 loss -0.0413 LR -0.0663 LKL 0.0250
epoch 7850 loss -0.0081 LR -0.0331 LKL 0.0250
epoch 7851 loss -0.1024 LR -0.1273 LKL 0.0248
epoch 7852 loss -0.0446 LR -0.0700 LKL 0.0254
epoch 7853 loss -0.0122 LR -0.0369 LKL 0.0248
epoch 7854 loss -0.0311 LR -0.0560 LKL 0.0249
epoch 7855 loss -0.0686 LR -0.0937 LKL 0.0251
epoch 7856 loss -0.0328 LR -0.0580 LKL 0.0253
epoch 7857 loss -0.0369 LR -0.0619 LKL 0.0251
epoch 7858 loss -0.0323 LR -0.0572 LKL 0.0250
epoch 7859 loss -0.0253 LR -0.0504 LKL 0.0251
epoch 7860 loss 0.0104 LR -0.0144 LKL 0.0248
epoch 7861 loss 0.0010 LR -0.0243 LKL 0.0253
epoch 7862 loss 0.0075 LR -0.0172 LKL 0.0247
epoch 7863 loss -0.0630 LR -0.0881 LKL 0.0251
epoch 7864 loss -0.0502 LR -0.0755 LKL 0.0253
epoch 7865 loss -0.0459 LR -0.0710 LKL 0.0251
epoch 7866 loss -0.0333 LR -0.0585 LKL 0.0252
epoch 7867 loss 0.0138 LR -0.0111 LKL 0.0249
epoch 7868 loss -0.0700 LR -0.0950 LKL

epoch 8029 loss -0.0537 LR -0.0789 LKL 0.0252
epoch 8030 loss 0.0357 LR 0.0105 LKL 0.0251
epoch 8031 loss -0.0426 LR -0.0677 LKL 0.0251
epoch 8032 loss -0.0295 LR -0.0544 LKL 0.0249
epoch 8033 loss -0.0422 LR -0.0670 LKL 0.0249
epoch 8034 loss -0.0397 LR -0.0647 LKL 0.0250
epoch 8035 loss -0.0619 LR -0.0867 LKL 0.0248
epoch 8036 loss -0.0704 LR -0.0953 LKL 0.0249
epoch 8037 loss 0.0217 LR -0.0033 LKL 0.0250
epoch 8038 loss -0.0613 LR -0.0861 LKL 0.0248
epoch 8039 loss -0.0828 LR -0.1079 LKL 0.0251
epoch 8040 loss -0.0779 LR -0.1029 LKL 0.0249
epoch 8041 loss -0.0372 LR -0.0621 LKL 0.0249
epoch 8042 loss -0.0575 LR -0.0825 LKL 0.0250
epoch 8043 loss -0.0234 LR -0.0480 LKL 0.0246
epoch 8044 loss -0.0241 LR -0.0488 LKL 0.0247
epoch 8045 loss -0.0526 LR -0.0775 LKL 0.0249
epoch 8046 loss -0.0606 LR -0.0857 LKL 0.0251
epoch 8047 loss -0.0417 LR -0.0667 LKL 0.0250
epoch 8048 loss -0.0607 LR -0.0857 LKL 0.0250
epoch 8049 loss -0.0753 LR -0.1006 LKL 0.0253
epoch 8050 loss -0.0070 LR -0.0319 LK

epoch 8210 loss -0.0651 LR -0.0907 LKL 0.0256
epoch 8211 loss -0.0776 LR -0.1033 LKL 0.0257
epoch 8212 loss 0.0245 LR -0.0007 LKL 0.0252
epoch 8213 loss 0.0043 LR -0.0210 LKL 0.0253
epoch 8214 loss -0.0893 LR -0.1144 LKL 0.0252
epoch 8215 loss -0.0061 LR -0.0316 LKL 0.0255
epoch 8216 loss -0.0834 LR -0.1089 LKL 0.0255
epoch 8217 loss -0.0276 LR -0.0528 LKL 0.0253
epoch 8218 loss -0.0416 LR -0.0669 LKL 0.0253
epoch 8219 loss -0.0355 LR -0.0610 LKL 0.0255
epoch 8220 loss -0.0842 LR -0.1098 LKL 0.0256
epoch 8221 loss -0.1024 LR -0.1279 LKL 0.0255
epoch 8222 loss -0.0445 LR -0.0699 LKL 0.0254
epoch 8223 loss -0.0700 LR -0.0954 LKL 0.0253
epoch 8224 loss -0.0706 LR -0.0959 LKL 0.0254
epoch 8225 loss -0.0582 LR -0.0834 LKL 0.0252
epoch 8226 loss -0.0800 LR -0.1054 LKL 0.0254
epoch 8227 loss -0.0624 LR -0.0879 LKL 0.0255
epoch 8228 loss -0.0760 LR -0.1014 LKL 0.0254
epoch 8229 loss -0.0666 LR -0.0919 LKL 0.0253
epoch 8230 loss -0.0265 LR -0.0520 LKL 0.0255
epoch 8231 loss -0.0330 LR -0.0582 L

KeyboardInterrupt: 